# Введение в методы сбора и обработки данных
## Добашнее задание к пятому уроку "MongoDB"

### Задание 1
Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные объявления с avito.ru в созданную БД (xpath/BS для парсинга на выбор)

#### Решение

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
import pprint

import warnings
warnings.filterwarnings('ignore')

In [2]:
def connection_to_db():
    """Возвращает коннектор к базе данных Mongo"""
    client = MongoClient('mongodb://127.0.0.1:27017')
    db = client['adverts']
    return db.adverts


def get_avito_content(target):
    """Делает запрос к avito и возвращает контент"""
    return requests.get(f'https://www.avito.ru/moskva?q={target}').text



def parse_content(content):
    """Парсит контент с avito"""
    
    items = []

    soup = bs(content, 'html.parser')
    divs = soup.findAll('div', 'item_table-wrapper')

    for div in divs:
        category = div.find('div', attrs={'class': 'data'}).contents[1].contents[0].strip()
        name = div.find('a', attrs={'class' :"item-description-title-link"}).text.strip()
        price = int(div.find('span', attrs={'class': 'price'})['content'])

        item = {
            'category': category,
            'name': name,
            'price': price
        }
        
        items.append(item)
    
    return items
        
def save_content_to_mongo(items):
    """Кладет контент в mongoDB"""
    adverts_db = connection_to_db()
    adverts_db.insert_many(items)

Вызываем функции и формируем db

In [3]:
# Скачиваем контент
content = get_avito_content('knife')

# Парсим контент
items = parse_content(content)

#Удалим старый контент из базы
adverts_db = connection_to_db()
adverts_db.drop()

# Сохраняем контент в базу
save_content_to_mongo(items)

Проверим что данные попали в базу

In [4]:
adverts_db.count()

50

### Задание 2 
Написать функцию, которая производит поиск и выводит на экран объявления с ценой меньше введенной суммы

#### Решение

In [5]:
def get_item_with_price_less(price_limit=None):
    """Выводит позиции товаров дешевле указанной пользователем суммы"""
    adverts_db = connection_to_db()
    
    if not price_limit:
        price_limit = input("Укажите максимальную сумму: ")
    
    print('\n')
    for p in adverts_db.find({"$and": [{"price": {"$gt": 0}}, {"price": {"$lt": int(price_limit)}}]}).sort("price"):
        pprint.pprint(p)

Вызовим функцию

In [6]:
get_item_with_price_less()

Укажите максимальную сумму: 500


{'_id': ObjectId('5d5101f11e9e453053b44ea5'),
 'category': 'Посуда и товары для кухни',
 'name': 'Ножеточка для ножниц и ножей Knife Sharpener.Новая',
 'price': 150}
{'_id': ObjectId('5d5101f11e9e453053b44eb4'),
 'category': 'Коллекционирование',
 'name': 'Значок брошь Нож охотничий hunting knife pin',
 'price': 300}
{'_id': ObjectId('5d5101f11e9e453053b44e89'),
 'category': 'Ремонт и строительство',
 'name': 'Мультитул 20 в 1 Super Knife',
 'price': 390}
{'_id': ObjectId('5d5101f11e9e453053b44e9b'),
 'category': 'Посуда и товары для кухни',
 'name': 'Точилка для ножей Bavarian edge Knife Sharpener',
 'price': 490}
